In [ ]:
import os
os._exit(0)

# **Inserindo o DataSet**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Carregando dataset em formato Pandas Dataframe a partir do arquivo CSV
df = pd.read_csv('/content/UCMF_FINAL.csv')

print(df.shape)
df.head()

(8043, 18)


,Peso,Altura,IMC,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,MOTIVO1,MOTIVO2,SEXO_F,SEXO_Indeterminado,SEXO_M
0,39.5,142.000000,20.0,9.60,0,85,50,2,1,1,0,90,1,2,4,0,0,1
1,26.0,112.000000,21.0,4.40,0,100,60,2,1,2,0,100,5,3,0,0,0,1
2,28.0,117.000000,20.0,5.89,0,110,65,4,1,1,0,80,5,2,3,1,0,0
3,24.5,122.000000,16.0,6.24,0,100,65,2,1,1,1,88,5,2,3,0,0,1
4,42.0,150.674161,18.5,15.50,0,100,60,1,0,1,0,52,5,3,0,0,0,1


- Visualização dos dados

In [3]:
print(df.shape)
df.head(10)

(8043, 18)


,Peso,Altura,IMC,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,MOTIVO1,MOTIVO2,SEXO_F,SEXO_Indeterminado,SEXO_M
0,39.5,142.000000,20.0,9.60,0,85,50,2,1,1,0,90,1,2,4,0,0,1
1,26.0,112.000000,21.0,4.40,0,100,60,2,1,2,0,100,5,3,0,0,0,1
2,28.0,117.000000,20.0,5.89,0,110,65,4,1,1,0,80,5,2,3,1,0,0
3,24.5,122.000000,16.0,6.24,0,100,65,2,1,1,1,88,5,2,3,0,0,1
4,42.0,150.674161,18.5,15.50,0,100,60,1,0,1,0,52,5,3,0,0,0,1
5,10.5,89.000000,13.0,2.72,0,90,60,2,0,1,0,95,5,2,3,1,0,0
6,47.0,145.000000,22.0,11.17,0,110,60,2,1,1,1,80,2,4,8,1,0,0
7,52.0,155.000000,22.0,17.26,0,140,90,8,1,1,0,70,4,2,0,1,0,0
8,45.0,155.962573,18.5,15.86,0,105,70,1,0,1,0,78,2,2,0,0,0,1
9,28.0,133.000000,16.0,8.43,0,140,100,8,1,1,1,100,0,4,6,0,0,1


In [4]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load dataset
data = df.copy()

# Separate classes
normal_data = data[data['NORMAL X ANORMAL'] == 1]
anormal_data = data[data['NORMAL X ANORMAL'] == 0]

# Count classes
normal_count = len(normal_data)
anormal_count = len(anormal_data)

# Determine how many samples need to be generated
samples_to_generate = normal_count - anormal_count

# Define GAN components with optimizations
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, activation='leaky_relu', input_dim=latent_dim))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='leaky_relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(data.shape[1], activation='tanh'))  # Use tanh for output scaling
    return model

def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Dense(512, activation='leaky_relu', input_shape=(data.shape[1],)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(256, activation='leaky_relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary output
    return model

# Compile models with Adam optimizer and adjusted learning rates
generator = build_generator(latent_dim=100)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build GAN
discriminator.trainable = False
gan_input = layers.Input(shape=(100,))
fake_data = generator(gan_input)
gan_output = discriminator(fake_data)
gan = keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop with optimizations
def train_gan(epochs=1200, batch_size=32):
    for epoch in range(epochs):
        # Train Discriminator
        idx_real = np.random.randint(0, normal_count, batch_size)
        real_samples = normal_data.iloc[idx_real].values

        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_samples = generator.predict(noise)

        # Add noise to real labels for stability
        d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)) * 0.9)  # One-sided label smoothing
        d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((batch_size, 1)))

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))


        # Print progress every 1000 epochs
        if epoch % 100 == 0:
            percent_complete = (epoch / epochs) * 100
            print(f"Epoch {epoch}/{epochs}, {percent_complete:.2f}% complete, D Loss: {d_loss_real[0] + d_loss_fake[0]}, G Loss: {g_loss}")

# Generate synthetic samples until balance is achieved
train_gan()

# Generate additional samples from generator and add them to anormal_data until balanced.
for _ in range(samples_to_generate):
    noise = np.random.normal(0, 1, (1, 100))
    generated_sample = generator.predict(noise)
    anormal_data.loc[len(anormal_data)] = generated_sample

# Combine datasets back together after balancing
balanced_data = pd.concat([normal_data, anormal_data], ignore_index=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/1200, 0.00% complete, D Loss: 1.2410353422164917, G Loss: [array(0.6525538, dtype=float32), array(0.6525538, dtype=float32), array(0.109375, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━

In [6]:

balanced_data.to_csv('UCMF_FINAL.csv', index=False)
from google.colab import files
files.download('UCMF_FINAL.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>